In [46]:
from pycalphad import Database, Model, calculate
import pycalphad.variables as v
import numpy as np
import sympy

In [47]:
dbf = Database('NI_AL_DUPIN_2001.TDB')

class NewModel(Model):
    def build_phase(self, dbe, phase_name, symbols, param_search):
        self.models['test'] = -13000*v.T*v.R*v.Y(phase_name, 0, 'AL')*v.Y(phase_name, 0, 'NI')
    bubbles = property(lambda self: self.models['test'] / 10)

# pycalphad needs custom properties to be defined for all phases, at the moment
# Maybe 'bubbles' has no meaning for FCC, so we return NaN
class FCCModel(Model):
    # Constant values need to be wrapped in sympy.S() so they are made symbolic
    bubbles = property(lambda self: sympy.S(np.nan))

In [48]:
result = calculate(dbf, ['AL', 'NI', 'VA'], ['LIQUID', 'FCC_L12'], model={'LIQUID': NewModel, 'FCC_L12': FCCModel},
                   output='bubbles', T=1500, P=101325, points={'LIQUID': [[0.9, 0.1], [0.2, 0.8]],
                                                               'FCC_L12': [[0.2, 0.8, 0.2, 0.8, 1], [0.4, 0.6, 0.6, 0.4, 1]]})

In [49]:
result['bubbles'].values[np.nonzero(result.Phase.values == 'LIQUID')]

array([-1459194.75, -2594124.  ])

In [50]:
result['bubbles'].values[np.nonzero(result.Phase.values == 'LIQUID')]

array([ nan,  nan])